<a href="https://colab.research.google.com/github/allisoncerna/Data-Science-Portfolio/blob/main/Project-1-Personal-Finance/Phase1_PersonalFinanceOptimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Personal Finance Leak Detection & Forecasting
**Author:** Allison Cerna  
**Target Role:** Data Scientist / Data Analyst  
**Tech Stack:** Python (Pandas, Matplotlib, Seaborn), Kaggle API

---

## Table of Contents
1. [Data Ingestion & API Configuration](#data-ingestion)
2. [Exploratory Data Analysis (EDA)](#eda)
3. [Data Cleaning & Health Audit](#data-cleaning)
4. [Feature Engineering: Identifying Leaks](#feature-engineering)
5. [Forecasting & Insights](#forecasting)

## Data Ingestion & API Configuration

In [2]:
import os
from google.colab import userdata

# Retrieving the token from your Colab Secrets
token = userdata.get('Kaggle_API_Token')

# Setting it as an 'Environment Variable' so Kaggle can find it
os.environ['Kaggle_API_Token'] = token

print("API Token successfully connected!")

API Token successfully connected!


In [3]:
import os
from google.colab import userdata

# 1. Setting my identity and key
os.environ['KAGGLE_USERNAME'] = "allisoncerna"
os.environ['KAGGLE_KEY'] = "KGAT_e4df6a29d15109d8ba885ab1f9ebfe44"

print("Credentials locked in. Testing connection...")

# 2. Testing the connection by searching for the kaggle dataset
!kaggle datasets list -s "personal-finance-ml-dataset"

Credentials locked in. Testing connection...
ref                                                           title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
miadul/personal-finance-ml-dataset                            Personal Finance ML Dataset                            1208244  2025-07-23 02:34:15.297000           4442         37  0.7058824        
parthpatel2130/realistic-loan-approval-dataset-us-and-canada  Realistic Loan Approval Dataset | US & Canada          1717268  2025-11-01 04:33:16.737000           3006         40  1.0              
elahehgolrokh/data-science-job-postings-with-salaries-2025    Data Science Job Postings with Salaries (2025)           39823  2025-09-07 12:51:21.687000           

In [4]:
# Downloading the data directly to Colab
!kaggle datasets download -d miadul/personal-finance-ml-dataset

# Unzipping and listing the files
!unzip \*.zip
!ls

Dataset URL: https://www.kaggle.com/datasets/miadul/personal-finance-ml-dataset
License(s): apache-2.0
  0% 0.00/1.15M [00:00<?, ?B/s]
100% 1.15M/1.15M [00:00<00:00, 638MB/s]
Archive:  personal-finance-ml-dataset.zip
  inflating: synthetic_personal_finance_dataset.csv  
personal-finance-ml-dataset.zip  synthetic_personal_finance_dataset.csv
sample_data


In [5]:
#Initial Data Audit (AKA Data Profiling)
import pandas as pd

# Loading the file into a DataFrame
df = pd.read_csv('synthetic_personal_finance_dataset.csv')

# 1. Checking the 'Shape' (How many rows and columns?)
print(f"Dataset Shape: {df.shape}")

# 2. Checking for missing values (Essential for 'Identifying Leaks')
print("\n--- Missing Values Report ---")
print(df.isnull().sum())

# 3. Looking at the first 5 rows to see the columns
display(df.head())

Dataset Shape: (32424, 20)

--- Missing Values Report ---
user_id                        0
age                            0
gender                         0
education_level                0
employment_status              0
job_title                      0
monthly_income_usd             0
monthly_expenses_usd           0
savings_usd                    0
has_loan                       0
loan_type                  19429
loan_amount_usd                0
loan_term_months               0
monthly_emi_usd                0
loan_interest_rate_pct         0
debt_to_income_ratio           0
credit_score                   0
savings_to_income_ratio        0
region                         0
record_date                    0
dtype: int64


,user_id,age,gender,education_level,employment_status,job_title,monthly_income_usd,monthly_expenses_usd,savings_usd,has_loan,loan_type,loan_amount_usd,loan_term_months,monthly_emi_usd,loan_interest_rate_pct,debt_to_income_ratio,credit_score,savings_to_income_ratio,region,record_date
0,U00001,56,Female,High School,Self-employed,Salesperson,3531.69,1182.59,367655.03,No,NaN,0.00,0,0.00,0.00,0.00,430,8.68,Other,2024-01-09
1,U00002,19,Female,PhD,Employed,Salesperson,3531.73,2367.99,260869.10,Yes,Education,146323.34,36,4953.50,13.33,1.40,543,6.16,North America,2022-02-13
2,U00003,20,Female,Master,Employed,Teacher,2799.49,1003.91,230921.21,No,NaN,0.00,0,0.00,0.00,0.00,754,6.87,Africa,2022-05-12
3,U00004,25,Male,PhD,Employed,Manager,5894.88,4440.12,304815.51,Yes,Business,93242.37,24,4926.57,23.93,0.84,461,4.31,Europe,2023-10-02
4,U00005,53,Female,PhD,Employed,Student,5128.93,4137.61,461509.48,No,NaN,0.00,0,0.00,0.00,0.00,516,7.50,Africa,2021-08-07


In [6]:
# Replace NaN with 'No Loan'
df['loan_type'] = df['loan_type'].fillna('No Loan')

# Checking to see if it worked
print(df['loan_type'].value_counts())
print("\nMissing values in loan_type:", df['loan_type'].isnull().sum())

loan_type
No Loan      19429
Home          3284
Education     3275
Business      3261
Car           3175
Name: count, dtype: int64

Missing values in loan_type: 0


## Exploratory Data Analysis (EDA)